In [9]:
#import libraries
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

In [16]:
#parameters
data_directory = "C:/Users/kashi/Documents/CMPE 258/ind_project/Data2/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json" 
model_weights_directory = 'C:/Users/kashi/Documents/CMPE 258/ind_project/Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [17]:
#read data
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) 
    #run for 151 batches
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   
        for batch_index in range(0, 16):   
            for i in range(0, 64):  
                #train one by one per character
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1
                #correct char is used os output from current to input of next
        yield X, Y

In [18]:
#model is built using 3 layers with 20% dropout
#Softmax activation is used in fully connected later to sample the character
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    #save the weights
    model.load_weights("C:/Users/kashi/Documents/CMPE 258/ind_project/Data/Model_Weights/Weights_80.h5", by_name = True)
    
    return model

In [19]:
def training_model(data, epochs = 90):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    #loss is calculated using categorical cross entropy and adam optimizer
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) 
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) 
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #save weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #calculating loss and accuarcy
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("C:/Users/kashi/Documents/CMPE 258/ind_project/Data2/log.csv", index = False)

In [20]:
#Run for 90 epochs
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             44544     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 256)             0         
_________________________________________________________________
lstm_5 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_5 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_6 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_6 (Drop

Batch: 113, Loss: 3.2293174266815186, Accuracy: 0.1552734375
Batch: 114, Loss: 3.0964322090148926, Accuracy: 0.1845703125
Batch: 115, Loss: 3.1147425174713135, Accuracy: 0.19140625
Batch: 116, Loss: 3.1298232078552246, Accuracy: 0.169921875
Batch: 117, Loss: 3.2201740741729736, Accuracy: 0.158203125
Batch: 118, Loss: 3.196624755859375, Accuracy: 0.173828125
Batch: 119, Loss: 3.2255167961120605, Accuracy: 0.1630859375
Batch: 120, Loss: 3.0671424865722656, Accuracy: 0.1767578125
Batch: 121, Loss: 3.1232192516326904, Accuracy: 0.1884765625
Batch: 122, Loss: 3.227503776550293, Accuracy: 0.146484375
Batch: 123, Loss: 3.1342954635620117, Accuracy: 0.1591796875
Batch: 124, Loss: 3.105271816253662, Accuracy: 0.162109375
Batch: 125, Loss: 3.085537910461426, Accuracy: 0.177734375
Batch: 126, Loss: 3.105081558227539, Accuracy: 0.1630859375
Batch: 127, Loss: 3.13724946975708, Accuracy: 0.1650390625
Batch: 128, Loss: 3.116724729537964, Accuracy: 0.1728515625
Batch: 129, Loss: 3.0819859504699707, Ac

Batch: 102, Loss: 2.0907950401306152, Accuracy: 0.392578125
Batch: 103, Loss: 2.360008478164673, Accuracy: 0.3564453125
Batch: 104, Loss: 2.086533546447754, Accuracy: 0.423828125
Batch: 105, Loss: 2.112945318222046, Accuracy: 0.4130859375
Batch: 106, Loss: 2.1913130283355713, Accuracy: 0.384765625
Batch: 107, Loss: 2.274351119995117, Accuracy: 0.3623046875
Batch: 108, Loss: 2.276242256164551, Accuracy: 0.388671875
Batch: 109, Loss: 2.193589925765991, Accuracy: 0.384765625
Batch: 110, Loss: 2.050849199295044, Accuracy: 0.390625
Batch: 111, Loss: 2.043163299560547, Accuracy: 0.4140625
Batch: 112, Loss: 2.2863523960113525, Accuracy: 0.3740234375
Batch: 113, Loss: 2.2744011878967285, Accuracy: 0.40625
Batch: 114, Loss: 2.1753218173980713, Accuracy: 0.3974609375
Batch: 115, Loss: 2.3078012466430664, Accuracy: 0.40234375
Batch: 116, Loss: 2.2834372520446777, Accuracy: 0.37109375
Batch: 117, Loss: 2.292055130004883, Accuracy: 0.3564453125
Batch: 118, Loss: 2.2240560054779053, Accuracy: 0.3886

Batch: 90, Loss: 1.8337507247924805, Accuracy: 0.4833984375
Batch: 91, Loss: 1.8117671012878418, Accuracy: 0.490234375
Batch: 92, Loss: 1.8598546981811523, Accuracy: 0.4833984375
Batch: 93, Loss: 1.8238954544067383, Accuracy: 0.4794921875
Batch: 94, Loss: 1.8023736476898193, Accuracy: 0.4755859375
Batch: 95, Loss: 1.7596499919891357, Accuracy: 0.48046875
Batch: 96, Loss: 1.8486276865005493, Accuracy: 0.4873046875
Batch: 97, Loss: 1.7688720226287842, Accuracy: 0.50390625
Batch: 98, Loss: 1.7215771675109863, Accuracy: 0.529296875
Batch: 99, Loss: 1.7031928300857544, Accuracy: 0.5107421875
Batch: 100, Loss: 1.7614145278930664, Accuracy: 0.490234375
Batch: 101, Loss: 1.8013815879821777, Accuracy: 0.494140625
Batch: 102, Loss: 1.6881704330444336, Accuracy: 0.4970703125
Batch: 103, Loss: 1.9337549209594727, Accuracy: 0.494140625
Batch: 104, Loss: 1.6624531745910645, Accuracy: 0.5234375
Batch: 105, Loss: 1.792923927307129, Accuracy: 0.4873046875
Batch: 106, Loss: 1.8363107442855835, Accuracy:

Batch: 79, Loss: 1.4744077920913696, Accuracy: 0.5986328125
Batch: 80, Loss: 1.5256123542785645, Accuracy: 0.5322265625
Batch: 81, Loss: 1.6802647113800049, Accuracy: 0.4853515625
Batch: 82, Loss: 1.6567569971084595, Accuracy: 0.4833984375
Batch: 83, Loss: 1.5202608108520508, Accuracy: 0.5634765625
Batch: 84, Loss: 1.6132087707519531, Accuracy: 0.5341796875
Batch: 85, Loss: 1.4718427658081055, Accuracy: 0.576171875
Batch: 86, Loss: 1.784909725189209, Accuracy: 0.4765625
Batch: 87, Loss: 1.5544137954711914, Accuracy: 0.564453125
Batch: 88, Loss: 1.705615758895874, Accuracy: 0.51953125
Batch: 89, Loss: 1.7271029949188232, Accuracy: 0.501953125
Batch: 90, Loss: 1.5863726139068604, Accuracy: 0.5361328125
Batch: 91, Loss: 1.549601435661316, Accuracy: 0.5322265625
Batch: 92, Loss: 1.6551504135131836, Accuracy: 0.5078125
Batch: 93, Loss: 1.5606129169464111, Accuracy: 0.54296875
Batch: 94, Loss: 1.5705361366271973, Accuracy: 0.5224609375
Batch: 95, Loss: 1.6378624439239502, Accuracy: 0.5029296

...

Batch: 151, Loss: 0.3837975263595581, Accuracy: 0.8662109375
Epoch 87/90
Batch: 1, Loss: 0.5444474220275879, Accuracy: 0.818359375
Batch: 2, Loss: 0.4869890511035919, Accuracy: 0.83984375
Batch: 3, Loss: 0.40469610691070557, Accuracy: 0.865234375
Batch: 4, Loss: 0.351454496383667, Accuracy: 0.8740234375
Batch: 5, Loss: 0.34958797693252563, Accuracy: 0.880859375
Batch: 6, Loss: 0.37865883111953735, Accuracy: 0.86328125
Batch: 7, Loss: 0.3848881721496582, Accuracy: 0.86328125
Batch: 8, Loss: 0.3446519374847412, Accuracy: 0.8740234375
Batch: 9, Loss: 0.41355133056640625, Accuracy: 0.859375
Batch: 10, Loss: 0.36458441615104675, Accuracy: 0.876953125
Batch: 11, Loss: 0.4304836392402649, Accuracy: 0.859375
Batch: 12, Loss: 0.4113362431526184, Accuracy: 0.85546875
Batch: 13, Loss: 0.33693012595176697, Accuracy: 0.8818359375
Batch: 14, Loss: 0.42062562704086304, Accuracy: 0.853515625
Batch: 15, Loss: 0.3694494962692261, Accuracy: 0.8857421875
Batch: 16, Loss: 0.36384117603302, Accuracy: 0.8896

Batch: 138, Loss: 0.3829786777496338, Accuracy: 0.87109375
Batch: 139, Loss: 0.3954075872898102, Accuracy: 0.861328125
Batch: 140, Loss: 0.4268040955066681, Accuracy: 0.853515625
Batch: 141, Loss: 0.41689592599868774, Accuracy: 0.8544921875
Batch: 142, Loss: 0.44445329904556274, Accuracy: 0.86328125
Batch: 143, Loss: 0.3932734429836273, Accuracy: 0.8681640625
Batch: 144, Loss: 0.40991127490997314, Accuracy: 0.8671875
Batch: 145, Loss: 0.3571699857711792, Accuracy: 0.8720703125
Batch: 146, Loss: 0.4117293357849121, Accuracy: 0.8681640625
Batch: 147, Loss: 0.3686464726924896, Accuracy: 0.873046875
Batch: 148, Loss: 0.3860078454017639, Accuracy: 0.8662109375
Batch: 149, Loss: 0.353371262550354, Accuracy: 0.8798828125
Batch: 150, Loss: 0.4141380786895752, Accuracy: 0.86328125
Batch: 151, Loss: 0.3621549606323242, Accuracy: 0.8779296875
Epoch 88/90
Batch: 1, Loss: 0.5259955525398254, Accuracy: 0.8359375
Batch: 2, Loss: 0.41809773445129395, Accuracy: 0.8525390625
Batch: 3, Loss: 0.3895290493

Batch: 125, Loss: 0.44829922914505005, Accuracy: 0.8369140625
Batch: 126, Loss: 0.4211477041244507, Accuracy: 0.8642578125
Batch: 127, Loss: 0.3469691872596741, Accuracy: 0.890625
Batch: 128, Loss: 0.42190322279930115, Accuracy: 0.8701171875
Batch: 129, Loss: 0.37717247009277344, Accuracy: 0.876953125
Batch: 130, Loss: 0.42121025919914246, Accuracy: 0.853515625
Batch: 131, Loss: 0.3630465865135193, Accuracy: 0.880859375
Batch: 132, Loss: 0.43040046095848083, Accuracy: 0.8603515625
Batch: 133, Loss: 0.3550199866294861, Accuracy: 0.875
Batch: 134, Loss: 0.3435009717941284, Accuracy: 0.875
Batch: 135, Loss: 0.37550246715545654, Accuracy: 0.87890625
Batch: 136, Loss: 0.38983726501464844, Accuracy: 0.8681640625
Batch: 137, Loss: 0.38596630096435547, Accuracy: 0.86328125
Batch: 138, Loss: 0.38071325421333313, Accuracy: 0.8701171875
Batch: 139, Loss: 0.39936694502830505, Accuracy: 0.859375
Batch: 140, Loss: 0.4025256931781769, Accuracy: 0.8662109375
Batch: 141, Loss: 0.43053266406059265, Accu

Batch: 112, Loss: 0.4308859705924988, Accuracy: 0.8427734375
Batch: 113, Loss: 0.3783634901046753, Accuracy: 0.8759765625
Batch: 114, Loss: 0.3876146376132965, Accuracy: 0.8662109375
Batch: 115, Loss: 0.4211353063583374, Accuracy: 0.853515625
Batch: 116, Loss: 0.4026329517364502, Accuracy: 0.8623046875
Batch: 117, Loss: 0.3997524678707123, Accuracy: 0.8701171875
Batch: 118, Loss: 0.3290001153945923, Accuracy: 0.892578125
Batch: 119, Loss: 0.3118559420108795, Accuracy: 0.904296875
Batch: 120, Loss: 0.3499596416950226, Accuracy: 0.880859375
Batch: 121, Loss: 0.41473251581192017, Accuracy: 0.861328125
Batch: 122, Loss: 0.3720933794975281, Accuracy: 0.8779296875
Batch: 123, Loss: 0.34128278493881226, Accuracy: 0.890625
Batch: 124, Loss: 0.3994724452495575, Accuracy: 0.873046875
Batch: 125, Loss: 0.42159032821655273, Accuracy: 0.8564453125
Batch: 126, Loss: 0.3931966722011566, Accuracy: 0.87890625
Batch: 127, Loss: 0.33456844091415405, Accuracy: 0.888671875
Batch: 128, Loss: 0.4285493493080

Batch: 99, Loss: 0.39516186714172363, Accuracy: 0.8642578125
Batch: 100, Loss: 0.40432459115982056, Accuracy: 0.861328125
Batch: 101, Loss: 0.38866233825683594, Accuracy: 0.87109375
Batch: 102, Loss: 0.3542167842388153, Accuracy: 0.880859375
Batch: 103, Loss: 0.3600591719150543, Accuracy: 0.8798828125
Batch: 104, Loss: 0.3551879823207855, Accuracy: 0.876953125
Batch: 105, Loss: 0.3967365622520447, Accuracy: 0.8642578125
Batch: 106, Loss: 0.3529924750328064, Accuracy: 0.8798828125
Batch: 107, Loss: 0.36706700921058655, Accuracy: 0.892578125
Batch: 108, Loss: 0.38543322682380676, Accuracy: 0.87109375
Batch: 109, Loss: 0.41224178671836853, Accuracy: 0.86328125
Batch: 110, Loss: 0.3797149062156677, Accuracy: 0.8671875
Batch: 111, Loss: 0.3846595585346222, Accuracy: 0.8701171875
Batch: 112, Loss: 0.375414103269577, Accuracy: 0.8798828125
Batch: 113, Loss: 0.40769246220588684, Accuracy: 0.859375
Batch: 114, Loss: 0.41470929980278015, Accuracy: 0.8642578125
Batch: 115, Loss: 0.403954893350601

In [22]:
#Gives better music quality
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,3.035434,0.198242
1,2,2.143913,0.405273
2,3,1.737826,0.514648
3,4,1.511529,0.560547
4,5,1.357965,0.592773
5,6,1.283942,0.611328
6,7,1.217910,0.616211
7,8,1.149021,0.638672
8,9,1.099961,0.649414
9,10,1.082244,0.645508
